In [148]:
import pandas as pd
import numpy as np
import pathlib as pl
import datetime as dt


p1 = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37')
p2 = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Partcipant 59_2022-11-21_10.37.29')


In [149]:
def import_finometer_txt(folder_path, save_csv=False, zero_time=False):
    """
    Imports raw data from the finopress finometer .txt file and returns a pandas dataframe, resahped from tall to wide format.
    The function will only work if there is only one .txt file in the folder.
    i: pathlib.Path object
    save_csv: boolean, if True, will save a csv file of the imported data into the participant's folder
    zero_time: boolean, if True, will convert the time column to a timedelta object and set the first timepoint to 0
    """

   
    if not isinstance(folder_path, pl.Path):#check if folder_path is a pathlib.Path object
        raise TypeError('folder_path must be a pathlib.Path object')
    elif not folder_path.exists(): #  and if it exists
        raise ValueError('folder_path does not exist')
    elif not folder_path.is_dir(): #  and is a directory 
        raise ValueError('folder_path is not a directory')
    elif len([x for x in folder_path.iterdir() if x.suffix == '.txt']) != 1:  # and if there is only one .txt file in the folder
        raise ValueError('''More than one .txt file found in folder, please ensure only one .txt file is present in the folder''')
    else:
        file = [x for x in folder_path.iterdir() if x.suffix == '.txt'][0]
    # read in the data from the .txt file
        df = pd.read_csv(
            file,
            sep=';',
            header=0,
            skiprows=8, # skip the first 8 rows
            skipfooter=1,
            engine='python',
            )

        # Drop the unnamed column at position 13
        df = df.drop(df.columns[13], axis=1)



        # set the 'Time (s)' column to a datetime object        
        df['Time (s)'] = pd.to_datetime(df['Time (s)'], format='%H:%M:%S.%f')
        # drop the year from the datetime object
        df['Time (s)'] = df['Time (s)'].dt.strftime('%H:%M:%S.%f')

        # check if the user wants to set the first timepoint to 0
        if zero_time:
            min_time = df['Time (s)'].min()
            df['Time Since 0'] = df['Time (s)'] - min_time
            df['Time Since 0'] = df['Time Since 0'].dt.total_seconds()
            df['']
        else:
            df = df.set_index('Time (s)')

        alligned = df.groupby(df.index)[df.columns[1:]].max().stack().to_frame().T
        alligned.index = [file.stem.split('_')[0]]

        if save_csv:
            alligned.to_csv(folder_path / f'imported data for {file.stem}.csv')
            print(f"CSV saved for {file.stem.split('_')[0]}")



        return alligned

In [150]:
a = import_finometer_txt(p1)

In [170]:
def import_finometer_demographics(file_path):
    '''
    A function to import the demographic data from a smartmedical/finopress
    finometer .txt file
    
    file_path =  should be a path to a .txt file, this can be string but should preferably be a pathlib object to allow for use on multiple operating
    systems
    '''

    if not isinstance(file_path, pl.PurePath):
        try:
            file_path = pl.Path(file_path)
        except TypeError:
            raise TypeError("""The 'file_path' must be a path object or a string that can be converted to a path object. Remember 'raw strings on windows devices""")
    
    try:
        with open(file_path, 'r') as file:
            data=[]
            for line in file:
                if line.startswith('Identification'):
                    data.append(line.strip().split(';'))
                    next_line = next(file).strip()
                    if next_line:
                        data.append(next_line.split(';'))
                    break
            
            if len(data) < 2:
                raise ValueError("File doesn't contain expected data")
            

            df = pd.DataFrame(data, columns= ['Participant ID', 'ID 2', 'Age (yrs)', 'Height (cm)', 'Weight (kg)', 'Gender', 'Procedure', 'Model number'])
            df = df.drop(0)
            df = df.drop('ID 2', axis = 1)


            return(df)      
                        

    except (TypeError, ValueError, FileNotFoundError) as err:
        print(type(err), err)
        return None


In [171]:
a = pl.Path(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Participant 1_2022-10-27_09.07.37\Participant 1_2022-10-27_09.07.37.txt')

b =import_finometer_demographics(a)

In [172]:
b

,Participant ID,Age (yrs),Height (cm),Weight (kg),Gender,Procedure,Model number
1,Participant 1,18,150,48,Female,,9715


In [173]:
df = pd.concat([import_finometer_demographics(f) for f in p1.parent.glob("**/*") if f.suffix == '.txt'], axis = 0, ignore_index=True)


In [174]:
df

,Participant ID,Age (yrs),Height (cm),Weight (kg),Gender,Procedure,Model number
0,Partcipant 59,20,162,61,Female,,9715
1,Partcipant 60,18,168,72,Female,,9715
2,Partcipant 64,18,164,57,Female,,9715
3,Participant 11,18,168,53,Female,,9715
4,Participant 1,18,150,48,Female,,9715


In [176]:
q_df = pd.read_csv(r'C:\Users\kevin.omalley\OneDrive - University of Limerick\Documents\GitHub\fino2py\data\Religiosity,+spirituality,+social+support+and+cardiovascular+responses+to+stress_February+15,+2023_06.09.csv')

In [179]:
with open(p1.parent/ 'qualtrics columns.txt', 'w') as file:
    for i,j in enumerate(q_df.columns):
        file.writelines(f'{i} = {j}\n')
